# Retrieval-augmented generation (RAG)


In [1]:
# !pip install langchain==0.0.331 langchainhub 

In [2]:
# !pip install openai==0.28.1

In [3]:
# !pip install faiss-cpu tiktoken

  Using cached tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
Using cached tiktoken-0.5.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)
Using cached regex-2023.10.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)


In [4]:
# !pip install python-dotenv 

In [5]:
# !pip install scikit-learn

  Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached scipy-1.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.2.0-py3-none-any.whl.metadata (10.0 kB)
Using cached scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.8 MB)
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Using cached scipy-1.11.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (36.4 MB)
Using cached threadpoolctl-3.2.0-py3-none-any.whl (15 kB)


In [1]:
!pip install youtube-transcript-api

In [17]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 294.4 kB/s eta 0:00:0031m4.8 MB/s eta 0:00:01


In [6]:
ls -la

total 28
drwxrwxr-x 3 wasit wasit  4096 Nov  8 14:22 ./
drwxrwxr-x 5 wasit wasit  4096 Nov  7 10:46 ../
-rw-rw-r-- 1 wasit wasit    66 Nov  7 10:57 .env
drwxrwxr-x 2 wasit wasit  4096 Nov  7 10:48 .ipynb_checkpoints/
-rw-rw-r-- 1 wasit wasit 10668 Nov  8 14:22 RAG.ipynb


In [6]:
# Set env var OPENAI_API_KEY or load from a .env file
import dotenv

dotenv.load_dotenv()

True

In [7]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo-16k", temperature=0)

In [8]:
loader = WebBaseLoader("https://www.history.com/topics/ancient-rome/ancient-rome")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(loader.load())

In [9]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
# vectorstore.save_local('vector.db')
# vectorstore=FAISS.load_local('vector.db', embeddings=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()

In [10]:
# from langchain.retrievers import SVMRetriever
# question="What are key facts from the articles?"
# svm_retriever = SVMRetriever.from_documents(splits, OpenAIEmbeddings())
# docs_svm = svm_retriever.get_relevant_documents(question)
# len(docs_svm)

In [11]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm
)

In [12]:
print(rag_chain.invoke("What are key facts from the articles?").content)

The Roman Empire was founded in 27 B.C. and was a vast and powerful domain. It gave rise to the culture, laws, technologies, and institutions that continue to define Western civilization. The articles were written or edited by the HISTORY.com editors, including Amanda Onion, Missy Sullivan, Matt Mullen, and Christian Zapata. The articles provide information about Ancient Rome, including its location and timeline. The content is regularly reviewed and updated for accuracy and fairness.


In [13]:
print(rag_chain.invoke("Why did the Roman empire fell?").content)

The Roman Empire fell due to a combination of factors including constant warfare, invasions by barbarian tribes, internal turmoil and political instability, widening wealth inequality, and the loss of provinces.


In [14]:
# from langchain.llms import GPT4All
# local_path = (
#     "./models/ggml-gpt4all-l13b-snoozy.bin"  # replace with your desired local file path
# )

# # Callbacks support token-wise streaming
# callbacks = [StreamingStdOutCallbackHandler()]

# # Verbose is required to pass to the callback manager
# llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

## Rubber Tire

In [22]:
from langchain.document_loaders import YoutubeLoader
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=-fusUxEPwsw", add_video_info=True
)
loader.load()

[Document(page_content="summertime is vacation time here in germany most car owners still prefer to drive even to distant holiday destinations when it comes to road safety tires are of course of paramount importance more than 50 million are sold each year in germany alone it's a multi-billion dollar business but where do these tires come from and under what conditions are they produced the tires of my family car are well past their prime they're getting old and are a bit brittle the need to replace them ahead of a family vacation has got me thinking about the origin and sustainability of tires on the market greifen stiebling a family-run business is one of the biggest independent dealers in western germany i'd like to buy a set of new summer tires and want some advice the store stocks the standard brands they're all good quality but i have a different question on my mind can i tell where a tire was produced is that possible yeah yes made in china japan the czech republic tires are clea

In [24]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(loader.load())
splits[-1]

Document(page_content="that good i'm curious to see how they perform on the road this time i've opted for sustainability i'm also confident they're safe too and with peace of mind i set off on my family vacation [Music] you", metadata={'source': '-fusUxEPwsw', 'title': 'Rubber tires — a dirty business | DW Documentary', 'description': 'Unknown', 'view_count': 5180478, 'thumbnail_url': 'https://i.ytimg.com/vi/-fusUxEPwsw/hq720.jpg', 'publish_date': '2019-09-17 00:00:00', 'length': 1706, 'author': 'DW Documentary'})

In [37]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever(k=10, lambda_mult=0.2)

In [38]:
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use five sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""
rag_prompt_custom = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | rag_prompt_custom | llm
)

In [39]:
print(rag_chain.invoke("Why does Rubber tires is a dirty business ?").content)

Rubber tires are considered a dirty business because of the environmental impact of their production and disposal. The production of natural rubber, which is used in tire manufacturing, has led to deforestation and the establishment of rubber plantations in Southeast Asia. Additionally, the burning of old tires as fuel contributes to air pollution. The retreading process for commercial vehicles and aircraft reduces the use of natural rubber, but the majority of vehicles, such as private cars, do not undergo this process. This results in a higher demand for new tires and further environmental consequences.


In [40]:
print(rag_chain.invoke("What does the video discuss about Thailand?").content)

The video discusses the role of the tire industry in Thailand and the difficulty in gaining access to information about the industry. It also mentions that rubber plantations are widespread in Thailand and that the industry plays an important role in the Thai economy. The video shows a visit to a rubber processing plant in Thailand and mentions that the workers there are Thai, not Cambodian. The video does not provide information about the working conditions or living conditions of people in the rubber industry in Thailand.


In [46]:
print(rag_chain.invoke("What was living condition of the workers in Thailand rubber farm?").content)

The living conditions of the workers in the Thailand rubber farm were described as decrepit huts on the edge of the plantation.


In [42]:
print(rag_chain.invoke("How much does the worker earn a month?").content)

The worker earns between 4,000 and 5,000 baht a month.


In [43]:
print(rag_chain.invoke("What was working condition of the worker in Thailand rubber processing plant?").content)

The workers in the Thailand rubber processing plant earn 5,000 baht a month, which is less than the minimum wage of 9,000 baht.


In [44]:
print(rag_chain.invoke("What are the worker daily working routine?").content)

The workers work 12 hours a day, 5 days a week, sometimes more. They start their night shift at 8 pm and end at about 5 in the morning.
